# Programatic Generation of Animations

This notebook demonstrates that we can have a program generate an animation and then send it to the player. Back when this was first developed, I had written a small program to mimic the [runway guard lights](https://aerosavvy.com/airport-lights/wig-wag-animation/) that sit at the entrance to many airplane runways. I figured it'd be a relatively simple and fun test.

## Approach (No Pun Intended)

For a super simple test, I have an strand of 11 LEDs on a board. The simplest test would simply be turning half of them to yellow and the other half off, displaying for a second or two, and then swapping. While there are some runway guard lights that do this, I wanted to mimic a more advanced behavior I've seen on some runways.

The behavior I want to mimic involves the following steps:
- Turn half of the light bar on yellow with a dim brightness, and the other half off
- Gradually turn the yellow LEDs from dim to bright
- Gradually turn the yellow LEDs from bright to dim
- Switch sides to "wig-wag"
- Repeat on the other side

I had originally written a program that does this by directly controlling the LEDs. Here's most of that code:

```
num_leds = 11
pixels = adafruit_dotstar.DotStar(board.SCLK, board.MOSI, num_leds, pixel_order=adafruit_dotstar.BRG)

left_side = range(6, 10 + 1, 1)
right_side = range(0, 5, 1)
yellow = (255, 255, 0)
off = (0, 0, 0)
steps_per_wig = 20
min_brightness = 0.0
max_brightness = 1.0
step_size = (max_brightness - min_brightness) / steps_per_wig

pixels.fill(off)
for i in range(20):
    # Which side is which
    if i % 2 == 0:
        yellow_side = left_side
        off_side = right_side
    else:
        yellow_side = right_side
        off_side = left_side
        
    # Fade in and out
    for j in off_side:
            pixels[j] = off
    for b in tuple(range(steps_per_wig + 1)) + tuple(range(steps_per_wig, 0 - 1, -1)):
        yellow_with_fade = yellow + (min_brightness + (b * step_size), )
        for j in yellow_side:
            pixels[j] = yellow_with_fade
        time.sleep(0.020)
pixels.fill(off)
```

Our approach is to adapt this program to generated `Pixel`s and `Frame`s to form an `Animation` and send that to the player.

Let's start off by taking some of these constants and reusing them. This assumes that you have an 11-LED strand for testing like I do:

In [1]:
num_leds = 11
left_side = range(6, 10 + 1, 1)
right_side = range(0, 6, 1)
steps_per_wig = 20
min_brightness = 0.0
max_brightness = 1.0
step_size = (max_brightness - min_brightness) / steps_per_wig

Notice that we copied the math constants and pixel assignments, but not the colors. We'll do that in a later step.

In the original program the outermost `for` loop served to tell us when the animation started and stopped. Each loop execution represented a single "fade in, fade out" phase. The loop counter being even or odd would tell us what side to do this on. For generating an animation, we only need one complete phase per side to have an animation that we can play as many times as we'd like. Let's import some libraries and change that loop to generate objects that our `Player` can use.

In [2]:
import sys


sys.path.append('../')

In [3]:
import dataclasses

from adafruit_dotstar_pi_animation.data import *


# Predefined pixels and we'll copy and modify
yellow_pixel = Pixel(red=255, blue=255, green=0, brightness=1.0)
off_pixel = Pixel(red=0, blue=0, green=0, brightness=1.0)

frames = []

# Each iteration represnts the frames to animate one side of the wig-wag.
for i in range(2):
    
    # Which side is which
    if i % 2 == 0:
        yellow_side = left_side
        off_side = right_side
    else:
        yellow_side = right_side
        off_side = left_side

    # Fade in and out
    # Each iteration represents one complete frame of animation
    for b in tuple(range(steps_per_wig + 1)) + tuple(range(steps_per_wig, 0 - 1, -1)):
        pixels = [None] * num_leds
        for j in off_side:
            pixels[j] = dataclasses.replace(off_pixel)
        yellow_with_fade_pixel = dataclasses.replace(yellow_pixel, brightness=min_brightness + (b * step_size))
        for j in yellow_side:
            pixels[j] = dataclasses.replace(yellow_with_fade_pixel)
            
        frame = Frame(pixels=pixels, display_ms=20)
        frames.append(frame)
        
# All of our frames are generated; let's add them to an animation that plays for 2 times
animation = Animation(frames=frames, loop_infinitely=False, pause_between_play_ms=0, max_plays=2)

Let's set up our player and try to play it:

In [13]:
import adafruit_dotstar
import board

from adafruit_dotstar_pi_animation.player import DotStarCircuitPlayer


pixels = adafruit_dotstar.DotStar(board.SCLK, board.MOSI, num_leds, pixel_order=adafruit_dotstar.BGR, auto_write=False)
player = DotStarCircuitPlayer(pixels)
player.load(animation)
player.play()

That worked! Here, we can take a look at each frame that was generated:

In [5]:
animation

Animation(frames=[Frame(display_ms=20, pixels=[Pixel(brightness=1.0, red=0, green=0, blue=0), Pixel(brightness=1.0, red=0, green=0, blue=0), Pixel(brightness=1.0, red=0, green=0, blue=0), Pixel(brightness=1.0, red=0, green=0, blue=0), Pixel(brightness=1.0, red=0, green=0, blue=0), Pixel(brightness=1.0, red=0, green=0, blue=0), Pixel(brightness=0.0, red=255, green=0, blue=255), Pixel(brightness=0.0, red=255, green=0, blue=255), Pixel(brightness=0.0, red=255, green=0, blue=255), Pixel(brightness=0.0, red=255, green=0, blue=255), Pixel(brightness=0.0, red=255, green=0, blue=255)]), Frame(display_ms=20, pixels=[Pixel(brightness=1.0, red=0, green=0, blue=0), Pixel(brightness=1.0, red=0, green=0, blue=0), Pixel(brightness=1.0, red=0, green=0, blue=0), Pixel(brightness=1.0, red=0, green=0, blue=0), Pixel(brightness=1.0, red=0, green=0, blue=0), Pixel(brightness=1.0, red=0, green=0, blue=0), Pixel(brightness=0.05, red=255, green=0, blue=255), Pixel(brightness=0.05, red=255, green=0, blue=255),

How many frames were generated?

In [6]:
len(animation)

84

How long does it take to play all frames in this animation?

In [7]:
animation.frame_total_time_ms

1680